This file contains the functions needed for Diameter_Depth_20mm and Diameter_Depth_12mm.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 10, 10
from matplotlib.pyplot import imshow
from numpy.linalg import norm
import rawpy
import PIL
from uncertainties import unumpy as unp
from uncertainties import ufloat
from uncertainties.umath import *

#dummyvalues
r_mond=1737000. #r_mond in meter
pixerr=10

In [3]:
#calculating the norm for arrays consisting of ufloat
def normunc(x):
    y=x**2
    #return unumpy.sqrt(sum(y))
    return sqrt(sum(y))

def arccosunc(x):
    yn=np.arccos(x.n)
    ys=np.abs(1/np.sqrt(1-x.n**2)*x.s)
    return ufloat(yn, ys)

def arcsinunc(x):
    yn=np.arcsin(x.n)
    ys=np.abs(1/np.sqrt(1-x.n**2)*x.s)
    return ufloat(yn, ys)

We know calculate Beta-Doppelstrich (angle between y-axis and shadowline) and then we calculate Beta = Beta-Strich + Beta-Doppelstrich. The definition of Beta-Strich can be found on the prictures in the proposal. We have as an input the coordinates of the craters and the diameter length and shadow length in pixels. 

In [4]:
''' 
    Referenzpunkt auf Schattenlinie wurde ausgewählt bei (-779.5894216038151, 1.8338318595135774)
'''
x=abs(-779.5894216038151)    # Abstand Origin-Schattengrenze auf Höhe der x-Achse
r_moon=2935.0    # berechnet mit radius_finder mit Fehler von +/-9
beta_strichstrich = 2*np.arcsin(x/(2*r_moon))
print('rad: ',beta_strichstrich)
print('deg: ',360/(2*np.pi)*beta_strichstrich)

rad:  0.266405305489
deg:  15.2638996444


In [5]:
def betafinder(crater_bild, Rad_pixel_jpg=2935.0):
    '''
        Findet beta strich auf skizze und hoehe in meter (H_mess) des kraters
        crater_bild, mitte_bild sind pixelkoordinaten von Krater und Mondmitte
    '''
    hoehe=crater_bild[1]; breite=crater_bild[0] 
    r_strich=sqrt(Rad_pixel_jpg**2-hoehe**2)
    val=breite/r_strich
    if np.abs(val)>=0.999: val=val/abs(val)
    beta_strich=arcsinunc(val)
    return beta_strich+beta_strichstrich, hoehe*r_mond/Rad_pixel_jpg

The following function calculates the real diameter and depth of the crater given the crater diameter and shadow length in pixels. Also given Beta, Phi (see proposal) and the factor which converts pixel to meter (pixtometer).

In [7]:
def diameter_depth(diameter_pix, shadow_pix, beta, Phi, H_mess, pixeltometer):
    #H_mess muss Fehlerbehaftet sein!!
    s_mess=shadow_pix*pixeltometer
    D_mess=diameter_pix*pixeltometer
    theta=arccosunc(H_mess/r_mond)#klein phi in proposal
    D_real=D_mess/sin(theta)
    depth=s_mess/sin(Phi)*sin(beta)
    return D_real, depth